#### 資料清洗:
1. 初步將食材名稱全部替換成整理好的詞庫(約3000個字詞)，存為新的key值newItem
2. 再來把名稱以同義詞庫取代，另存key值分為ingredient及seasoning

In [1]:
import time
import json
import pandas as pd
import collections

In [2]:
with open('./foodBank/original_total_recipe.json', 'r', encoding='utf-8') as f:
    content = json.loads(f.read())

In [ ]:
# 用regular expression檢查食材名稱的雜亂程度
import re
big_set = set()
for i in content:
    for j in i['item']:
        big_set.add(j[0])

In [ ]:
# 測試預期的名稱pattern能對應到甚麼名稱
pattern = r'.*調味料.*'
for e in big_set:
    match =  re.findall(pattern,e)
    if match != []:
        print(match)

In [3]:
# 將全部食材合在一個文件進行名稱清洗
with open('./foodBank/all_ingredient.txt', 'r', encoding='utf-8') as f: 
    ingrediant_wk = f.read().splitlines()
    ingrediant_wk.sort(key=len,reverse=True)
# print(ingrediant_wk)

In [4]:
# 初步清洗，方法1 三重迴圈，較慢，約300秒

start_time = time.time()

# add new_list into original recipe
print(len(content))
# 記錄沒配對到的食材名稱還有數量
losing_list=[]
#因名稱對不上導致lose的食譜量
losing_count=0
index=0
for k in content:
    
    new_ingredient_list =[]
    item_list = k['item']  # [[a,1],[b,2],[ind,unit]]
    
    for each in range(len(item_list)):
        original_item = item_list[each][0].lower()
        original_unit = item_list[each][1]
        
        switch = True
        # go through the ingredient bank to match the name
        for word in ingrediant_wk:
            if word in original_item:
                new_ingredient_list.append([word,original_unit])
                switch = False
                break 
                    
        if switch == True:
            losing_list.append(original_item)
        
    if len(item_list) != len(new_ingredient_list):
        losing_count += 1
    
    index +=1
    
    if index % 10000 == 0:
        print(f'\r Data completed: {index}',end='')
    
    
    k['newItem'] = new_ingredient_list

    
print(f'number of the ingredient name unmatched: {len(losing_list)}')
print(f'number of recipe may losing: {losing_count}')


end_time = time.time()
print('Updating completed!')
print(f'Time used: {end_time-start_time}')

242829
 Data completed: 240000number of the ingredient name unmatched: 25072
number of recipe may losing: 19590
Updating completed!
Time used: 245.27793097496033


In [5]:
# 檢查漏掉的名稱是否需要增補進詞庫
check_ = {}
for i in losing_list:
    if i not in check_:
        check_[i] = 1
    else:
        check_[i] += 1
print(sorted(check_.items(), key= lambda x:x[1]))

[('愛之味條瓜罐頭', 1), ('地爪粉', 1), ('（鍋物）丸類', 1), ('軟施', 1), ('袋子或保鮮盒', 1), ('《備料的部分》--------', 1), ('3種口味', 1), ('炸酥粉', 1), ('ｂ：胚芽粉', 1), ('chocobaby', 1), ('鋁箔容器', 1), ('食用銀粉少許', 1), ('無藥濟食安木頭', 1), ('大釘針', 1), ('槌子', 1), ('凌檬汁', 1), ('飛利浦料理魔法棒hr1267', 1), ('無鋁泡達粉', 1), ('其他顏料', 1), ('牙籤(非必需)', 1), ('真珠粉', 1), ('杏乾', 1), ('红蘿菠', 1), ('任何模型皆可', 1), ('nutmeg粉', 1), ('五穀粒', 1), ('適當的模具', 1), ('紙杯直徑9*高6*', 1), ('本種', 1), ('八吋圓形模', 1), ('調味包浦燒鰻', 1), ('液體調味料', 1), ('曼越莓乾', 1), ('桂格穀珍十穀粉', 1), ('夾鏈袋1l', 1), ('大福皮', 1), ('13 x 9 吋烤盤', 1), ('陶磁製小玩偶', 1), ('粘粉', 1), ('西班牙風亁火煺', 1), ('鬼頭刀', 1), ('塑膠繩', 1), ('曼越莓', 1), ('瓦斯爐版', 1), ('電鍋版', 1), ('三寶粉(可不加)', 1), ('製作餡料', 1), ('主材料~', 1), ('其他喜歡的材料', 1), ('食用銀粉', 1), ('20公分鑄鐵媽媽鍋', 1), ('6吋鋁箔模', 1), ('彩球榖片', 1), ('三明治袋&緞帶', 1), ('其他裝飾用材料', 1), ('辣', 1), ('奇雅籽', 1), ('內餡部份', 1), ('2. 內餡', 1), ('模具一大一小', 1), ('15公分方型底分離', 1), ('［器皿］適當容器', 1), ('裝飾~', 1), ('potatoe', 1), ('原味戚風配方', 1), ('透明單張紙', 1), ('logo貼紙', 1), ('有花紋的透明自黏袋', 1), ('透明包裝盒', 1), ('手提紙盒', 1)

In [6]:
# 計算改名完，食材名稱為空list的食譜
count = 0
for i in content:
    if i['newItem'] == []:
#         print(f"{i['recipe']}: {i['url']}")
#         print(i['item'])
#         print(i['newItem'])
#         print("==================================================================")
        count += 1
print(f'number of recipe needed to delete: {count}')

number of recipe needed to delete: 528


In [9]:
# 檢查
for i in range(100):
    k = content[i]['newItem']
    print(k)

[['蛋', '2顆'], ['水', '50ml'], ['味霖', '1大匙'], ['鰹魚粉', '1小匙'], ['太白粉', '2小匙']]
[['草莓', '10顆'], ['白巧克力', '適量'], ['黑巧克力', '適量'], ['煉奶', '適量'], ['巧克力', '少許']]
[['低筋麵粉', '2杯'], ['泡打粉', '1大匙'], ['糖', '1/4杯'], ['鮮奶油', '3/4杯'], ['奶油', '1/4杯'], ['柳橙', '1個'], ['藍莓乾', '1/2杯']]
[['羊小排', '500g'], ['橄欖油', '100ml'], ['第戎芥末', '3大匙'], ['迷迭香', '1大匙']]
[['高筋麵粉', '360克'], ['鹽', '7克'], ['砂糖', '42克'], ['鮮奶', '213'], ['酵母', '5.5克'], ['無鹽奶油', '65克'], ['百里香', '1茶匙'], ['胡椒', '1/2茶匙'], ['香腸', '數條']]
[['紅豆', '1杯'], ['紫米', '1杯'], ['湯圓', '300克'], ['糖', '適量']]
[['馬鈴薯', '1-2顆'], ['食用油', '適量']]
[['丸子', '500g'], ['香菇', '6朵'], ['紅蘿蔔', '1條'], ['甜豆', '100g'], ['蔥', '2根'], ['竹筍', '1根'], ['蠔油', '4大匙'], ['糖', '2匙']]
[['草莓', '適量'], ['煉乳', '適量']]
[['三層肉', '約600克'], ['穀盛壽喜燒', '1瓶'], ['水', '2瓶'], ['米酒', '3大匙'], ['蔥', '3根量']]
[['南瓜', '1/3個'], ['雞腿', '100g'], ['玉米', '適量'], ['橄欖油', '1大匙'], ['飯', '2碗'], ['洋葱', '1/4個'], ['胡椒粉', '少許'], ['米酒', '少許']]
[['小米', '一米杯'], ['米', '半米杯'], ['水', '七米杯'], ['桂圓', '適量'], ['糖', '各人喜好'], ['水', '一米杯']]
[

In [7]:
# 刪除食材為空list的食譜
new_content = []
for i in content:
    if i['newItem'] == []:
        continue
    else:
        new_content.append(i)
print(len(new_content))

242301


In [11]:
242829-242301

528

In [8]:
#準備將食材分成食材及調味料
# 讀取同義詞庫
df = pd.read_excel("./foodBank/ingredient_unified_name.xlsx",header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,88,89,90,91,92,93,94,95,96,97
0,飯,壽司飯,稀飯,隔夜飯,薑黃飯,十穀飯,醋飯,炒飯,五穀飯,油飯,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,麵,全麦面,蝦子麵,老麵,陽春麵,炒麵,牛肉麵,公仔麵,燕麥麵,涼麵,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,義大利麵,筆管麵,義大利麵,義大利直麵,spaghetti,Spaghetti,螺旋麵,意大利麵,千層麵,貝殼麵,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,泡麵,花雕雞麵,麻油雞麵,味味麵,蒸煮麵,王子麵,即食麵,科學麵,辛拉麵,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,素麵,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# 確認詞庫長度
len(df)

675

In [10]:
df2 = pd.read_excel("./foodBank/seasoning_unified_name.xlsx",header=None)
df2.head()

,0,1,2,3,4,5,6,7,8,9,...,28,29,30,31,32,33,34,35,36,37
0,水,飲用水,生飲水,氣泡水,礦泉水,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,胡椒,黑椒,楜椒,糊椒,胡椒粒,多香果,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,胡椒粉,山椒粉,黑椒粉,古月粉,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,山椒,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,馬告,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# 確認詞庫長度
len(df2)

217

In [12]:
# 將dataframe直接轉成dictionary型式，加快後面同義詞庫取代的作業時間
ing_list = {}
for i in range(len(df)):    
    s = df.iloc[i].dropna().tolist()
    for i in s:
        ing_list[i] = s[0]
    
sea_list = {}
for i in range(len(df2)):
    s = df2.iloc[i].dropna().tolist()
    for i in s:
        sea_list[i] = s[0]


In [33]:
sea_list

{'水': '水',
 '飲用水': '水',
 '生飲水': '水',
 '氣泡水': '水',
 '礦泉水': '水',
 '胡椒': '胡椒',
 '黑椒': '胡椒',
 '楜椒': '胡椒',
 '糊椒': '胡椒',
 '胡椒粒': '胡椒',
 '多香果': '胡椒',
 '胡椒粉': '胡椒粉',
 '山椒粉': '胡椒粉',
 '黑椒粉': '胡椒粉',
 '古月粉': '胡椒粉',
 '山椒': '山椒',
 '馬告': '馬告',
 '鰹魚粉': '鰹魚粉',
 '鰹魚調味料': '鰹魚粉',
 '鰹魚風味調味料': '鰹魚粉',
 '堅魚粉': '鰹魚粉',
 '柴魚粉': '鰹魚粉',
 '鏗魚粉': '鰹魚粉',
 '柴魚精': '鰹魚粉',
 '鰹魚調味粉': '鰹魚粉',
 '魚粉': '鰹魚粉',
 '炒菜油': '炒菜油',
 '調和油': '炒菜油',
 '調合油': '炒菜油',
 '健康油': '炒菜油',
 '食油': '炒菜油',
 '花生油': '炒菜油',
 '沙拉油': '炒菜油',
 '炸油': '炒菜油',
 '大豆油': '炒菜油',
 '色拉油': '炒菜油',
 '豆油': '炒菜油',
 '炒油': '炒菜油',
 '玄米油': '炒菜油',
 '米糠油': '炒菜油',
 '米油': '炒菜油',
 '食用油': '炒菜油',
 '油': '炒菜油',
 '植物油': '植物油',
 '蔬菜油': '植物油',
 '芥花籽油': '植物油',
 '芥花油': '植物油',
 '葵花油': '植物油',
 '葡萄籽油': '植物油',
 '亞麻仁油': '植物油',
 '葵花籽油': '植物油',
 '菜籽油': '植物油',
 '菜油': '植物油',
 '亞麻籽油': '植物油',
 '酪梨油': '植物油',
 '玉米油': '植物油',
 '粟米油': '植物油',
 '橄欖油': '植物油',
 '橄榄油': '植物油',
 '橄欄油': '植物油',
 '欖油': '植物油',
 '茶油': '植物油',
 '茶籽油': '植物油',
 '苦茶油': '植物油',
 '芥籽油': '植物油',
 'olive oil': '植物油',
 '香草油': '香草油',
 '雲尼拿油': '香草油

In [13]:
#4. 新新新方法，結合前面將詞庫做成字典方式降低查找資料的時間複雜度
start_time = time.time()
i = 0
for k in new_content:
    new_seasoning_list = []
    new_ingredient_list = []
    item_list = k['newItem'] 
    for item_name,item_unit in item_list:

        if item_name in ing_list:
                # print(f'{item_name} matched.')
                replace_name = ing_list[item_name]
                new_ingredient_list.append([replace_name,item_unit])
                continue
        elif item_name in sea_list:
                # print(f'{item_name} matched.')
                replace_name = sea_list[item_name]
                new_seasoning_list.append([replace_name,item_unit])
                continue
        else:
            print(f'Error: {item_name} is not matched.')
    
    k['ingredient'] = new_ingredient_list
    k['seasoning'] = new_seasoning_list
    
    i += 1
    if i % 10000 == 0:
        print(f'\r Data completed: {i}',end='')
    
end_time = time.time()
print('\nUpdating completed!')
print(f'Time used: {end_time-start_time}')

 Data completed: 240000
Updating completed!
Time used: 2.0808725357055664


In [ ]:
#1. 舊方法，迴圈太多太耗時間
start_time = time.time()
i = 0
for k in new_content:
    new_seasoning_list = []
    new_ingredient_list = []
    item_list = k['newItem'] 
    for each in range(len(item_list)):
        item_name = item_list[each][0]
        item_unit = item_list[each][1]
        
        switch = True
        for i in range(len(df)):
            foodName_oneRow = df.iloc[i].dropna()
            for j in foodName_oneRow:
                if item_name == j:
                    new_ingredient_list.append([foodName_oneRow[0],item_unit])
                    switch = False
                    break
        
        if switch == True:
            for i in range(len(df2)):
                foodName_oneRow = df2.iloc[i].dropna()
                for j in foodName_oneRow:
                    if item_name == j:
                        new_seasoning_list.append([foodName_oneRow[0],item_unit])
    
    k['ingredient'] = new_ingredient_list
    k['seasoning'] = new_seasoning_list
    
    i += 1
    if i % 2000 == 0:
        print(f'Data completed: {i}')
    
end_time = time.time()
print('Updating completed!')
print(f'Time used: {end_time-start_time}')

In [ ]:
#2. 新方法，直接用df比對
start_time = time.time()
i = 0
for k in new_content:
    new_seasoning_list = []
    new_ingredient_list = []
    item_list = k['newItem'] 
    
    for each in range(len(item_list)):
        item_name = item_list[each][0]
        item_unit = item_list[each][1]
        # 先用食材詞庫比對，方法是用食譜的食材名稱在df搜尋，若有找到相符的名稱，則找出該名稱所在的row中第一個值替換原有食材名稱
        try:
            df_ = df[ df == item_name].fillna(0).replace(item_name,1).sum(axis=1)
            index = df_[df_ > 0].index.tolist()
            if index != []:
                replace_name = df.iloc[index[0],0]
                new_ingredient_list.append([replace_name,item_unit])
            # index == [],表示前一個詞庫沒比對到，再利用調味料詞庫比對一次，重複上面作法
            else:
                df_ = df2[ df2 == item_name].fillna(0).replace(item_name,1).sum(axis=1)
                index = df_[df_ > 0].index.tolist()
                if index != []:
                    replace_name = df2.iloc[index[0],0]
                    new_seasoning_list.append([replace_name,item_unit])
                else:
                    print(f'Error: {item_name} is not matched.')
        except Exception as e:
            print(f'{e}: {item_name}')
        
    k['ingredient'] = new_ingredient_list
    k['seasoning'] = new_seasoning_list

    
    i += 1
    if i % 2000 == 0:
        print(f'Data completed: {i}')
    
end_time = time.time()
print('Updating completed!')
print(f'Time used: {end_time-start_time}')

In [ ]:
#3. 新新方法，把df每個row轉成list比對
start_time = time.time()
i = 0
for k in new_content:
    new_seasoning_list = []
    new_ingredient_list = []
    item_list = k['newItem'] 
    for item_name,item_unit in item_list:
     
        switch = True
        for i in range(len(df)):
            foodName_oneRow = df.iloc[i].dropna().tolist()
            if item_name in foodName_oneRow:
                # print(f'{item_name} matched.')
                new_ingredient_list.append([foodName_oneRow[0],item_unit])
                switch = False
                break
        
        if switch == True:
            for i in range(len(df2)):
                foodName_oneRow = df2.iloc[i].dropna().tolist()
                if item_name in foodName_oneRow:
                    # print(f'{item_name} matched.')
                    new_seasoning_list.append([foodName_oneRow[0],item_unit])
                    switch = False
                    break
        
        if switch == True:
            print(f'Error: {item_name} is not matched.')
    
    k['ingredient'] = new_ingredient_list
    k['seasoning'] = new_seasoning_list
    
    i += 1
    if i % 500 == 0:
        print(f'\r Data completed: {i}',end='')
    
end_time = time.time()
print('Updating completed!')
print(f'Time used: {end_time-start_time}')

In [19]:
# 檢查內容
for i in range(100):
    k = new_content[i]['ingredient']
    print(k)

[['雞蛋', '2顆'], ['太白粉', '2小匙']]
[['草莓', '10顆'], ['牛奶巧克力', '適量'], ['巧克力', '適量'], ['煉乳', '適量'], ['巧克力', '少許']]
[['麵粉', '2杯'], ['發粉', '1大匙'], ['鮮奶油', '3/4杯'], ['柳橙', '1個'], ['藍苺', '1/2杯']]
[['羊肉', '500g'], ['迷迭香', '1大匙']]
[['麵粉', '360克'], ['牛奶', '213'], ['酵母', '5.5克'], ['百里香', '1茶匙'], ['香腸', '數條']]
[['紅豆', '1杯'], ['紫米', '1杯'], ['湯圓', '300克']]
[['馬鈴薯', '1-2顆']]
[['丸子', '500g'], ['香菇', '6朵'], ['紅蘿蔔', '1條'], ['豌豆', '100g'], ['蔥', '2根'], ['筍', '1根']]
[['草莓', '適量'], ['煉乳', '適量']]
[['三層肉', '約600克'], ['蔥', '3根量']]
[['南瓜', '1/3個'], ['雞腿', '100g'], ['玉米', '適量'], ['飯', '2碗'], ['洋蔥', '1/4個']]
[['米', '一米杯'], ['米', '半米杯'], ['龍眼', '適量']]
[['起士', '250g'], ['蛋黃', '3顆'], ['雞蛋', '2顆'], ['餅乾', '10條'], ['oreo', '5個'], ['草莓', '6顆']]
[['雞肉', '半隻'], ['紅蘿蔔', '1條']]
[['小番茄', '600g'], ['香草', '半湯匙']]
[['豬肉', '8 小片'], ['豆芽菜', '80-100 g']]
[['豬肉', '200g'], ['芋頭', '2斤'], ['油蔥酥', '4~5大匙'], ['地瓜粉', '120g~150g'], ['韮菜', '適量']]
[['雞蛋', '2顆'], ['蝦', '半斤300g'], ['海苔', '一張'], ['紅蘿蔔', '2小細條']]
[['透抽', '1尾'], ['薑', '一小塊'], ['大蒜

In [14]:
# write into new json
with open('./foodBank/total_recipe_0924_ingredient_cleaned.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(new_content))

In [36]:
# 檢視同義字替換
ing_col = collections.Counter()
sea_col = collections.Counter()
for k in new_content:
    for i in k['ingredient']:
        ing_col.update([i[0]])
    for j in k['seasoning']:
        sea_col.update([j[0]])

print('總食材種類: {}'.format(len(ing_col.keys())))
print('總調味品種類: {}'.format(len(sea_col.keys())))

總食材種類: 675
總調味品種類: 214


In [37]:
sea_col.most_common(10)

[('鹽', 97698),
 ('砂糖', 95141),
 ('水', 72702),
 ('醬油', 56119),
 ('奶油', 39202),
 ('料理酒', 34974),
 ('胡椒', 31106),
 ('胡椒粉', 30348),
 ('植物油', 21854),
 ('炒菜油', 20472)]

In [42]:
ing_col.most_common(10)

[('大蒜', 61323),
 ('雞蛋', 60902),
 ('麵粉', 52352),
 ('蔥', 45591),
 ('薑', 36303),
 ('洋蔥', 36141),
 ('牛奶', 34296),
 ('豬肉', 34273),
 ('紅蘿蔔', 32358),
 ('辣椒', 27831)]